In [7]:
# Vanilla Python version. Before optimizing, get a vanilla code set in Python.

# The optimization path will probably look something like this:
'''
    - Write vanilla Python code.
            - Wall Time: 2m41s
    - Make easy Python improvements (e.g, @jit, numbafy)
            - @jit: Wall Time: 17.9s
            - @jit on Mac (no VM): 11.3s
            - add numpy: 2s?
            - remove print: like 5s?
    - Make harder Python improvements (not sure what they are a priori)
    - Write code in C if it makes sense
    - Figure out what PEP8 is, and make sure I'm using it
'''

# Throw in numba jit compile.
from numba import jit

# Add more numpy stuff.
import numpy as np

# Setup variables.
alphas = np.arange(0.5,1,0.1)
ns = np.arange(50,550,50)

# Define a function to take an (alpha, n, beta) tuple, and return the bias for the estimate.

# Store this as a lambda function so it can compile the operation once. (?)
estimate = lambda sumx, sumy, sumxsq, sumxy, n : (sumxy - (n-1)*(sumx/(n-1))*(sumy/(n-1)))/(sumxsq - (n-1)*math.pow((sumx/(n-1)),2));

@jit
def bias(alpha, n, beta):
        sumx = 0
        sumy = 0
        sumxy = 0
        sumxsq = 0
        x = rand()*100
        for i in range(n):
            sumx += x
            sumxsq += math.pow(x,2)
            y = beta + alpha*x + rand()
            sumxy += x*y
            sumy += y
            x = y
        val = estimate(sumx, sumy, sumxsq, sumxy, n) - alpha
        return val

# Define a function to take an (alpha, n, beta) tuple, estimate 10,000 times, and then average to get the expected bias.
from numpy.random import normal as rand
import math

@jit
def calcbias(alpha, n, beta):
    biases = np.zeros(10000)
    for i in range(10000):
        biases[i] = bias(alpha,n,beta)
    avg = np.mean(biases)
    return avg

# Define a function to call the above on the right sets of params, and return the biases.
@jit
def main():
    results = np.zeros(50)
    for i in range(5):
        for j in range(10):
            results[10*i+j] = calcbias(alphas[i],ns[j],1)
    return results
    print(results)

In [8]:
%time main()

CPU times: user 11.5 s, sys: 54.8 ms, total: 11.5 s
Wall time: 11.6 s


array([-0.00786783, -0.00505544, -0.0039289 , -0.0036889 , -0.00292907,
       -0.00212471, -0.00231391, -0.00216181, -0.00214432, -0.00173809,
       -0.00878167, -0.0054512 , -0.00422871, -0.00331306, -0.00340195,
       -0.00306514, -0.00210149, -0.00235711, -0.0023369 , -0.0019553 ,
       -0.01028647, -0.00637818, -0.00500796, -0.00405891, -0.00316635,
       -0.00338665, -0.00317961, -0.00267724, -0.00244083, -0.00202057,
       -0.01437032, -0.00775237, -0.00525223, -0.00445739, -0.00386865,
       -0.00330584, -0.0030391 , -0.00298755, -0.00278921, -0.00267807,
       -0.06853   , -0.01269166, -0.00680428, -0.00559531, -0.00442615,
       -0.00416332, -0.00360956, -0.00330238, -0.00285154, -0.00307589])